In [ ]:
!pip install import-ipynb
import import_ipynb

from CVAE_ImbalanceGenerator_MNIST import *
from MNIST_CVAE_Sampler_Gen import *

  Using cached import-ipynb-0.1.3.tar.gz (4.0 kB)
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=5553d5a8beed2960a8bfe60295ec802c6a980b5f5601a1884bb4f00d4b39d9ec
  Stored in directory: c:\users\anush goel\appdata\local\pip\cache\wheels\b1\5e\dc\79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.


importing Jupyter notebook from CVAE_ImbalanceGenerator_MNIST.ipynb


You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.


importing Jupyter notebook from MNIST_CVAE_Sampler_Gen.ipynb

You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.


-------------------
SETTING: Half-Split Imbalance
-------------------
epoch: 1 epoch_loss: 166.58161136881512
epoch: 2 epoch_loss: 136.826664444987
epoch: 3 epoch_loss: 127.52346910807292
epoch: 4 epoch_loss: 122.16022704264323
epoch: 5 epoch_loss: 118.52762172037761
epoch: 6 epoch_loss: 115.92709948730469
epoch: 7 epoch_loss: 114.19473244628907
epoch: 8 epoch_loss: 112.4619385904948
epoch: 9 epoch_loss: 111.18029829915365
epoch: 10 epoch_loss: 109.59101739095053
epoch: 11 epoch_loss: 108.39028171386718
epoch: 12 epoch_loss: 107.1648370320638
epoch: 13 epoch_loss: 106.25907088216145
epoch: 14 epoch_loss: 105.7184390991211
epoch: 15 epoch_loss: 104.97826937662761
epoch: 16 epoch_loss: 104.62331595865885
epoch: 17 epoch_loss: 104.1849951538086
epoch: 18 epoch_loss: 103.49219366861979
epoch: 19 epoch_loss: 103.29925423990885
epoch: 20 epoch_loss: 102.9153200358073
epoch: 21 epoch_loss: 102.498538671875
epoch: 22 epoch_loss: 102.33979076334636
epoch: 23 epoch_loss: 101.9237311319987
epoch:

In [ ]:
!pip install seaborn
!pip install imblearn
!pip install sklearn

You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.



  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.9.0-py3-none-any.whl (199 kB)



You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Anush Goel\anaconda4\python.exe -m pip install --upgrade pip' command.


In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F

import random
from copy import deepcopy

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np

from imblearn.metrics import (geometric_mean_score, sensitivity_score, 
                              specificity_score)
from sklearn.metrics import (balanced_accuracy_score, precision_score, 
                             recall_score, f1_score)


import scipy as sc
import matplotlib.style

params = {'legend.fontsize': 14,
          'axes.labelsize': 14,
          'axes.titlesize': 14,
          'xtick.labelsize' :14,
          'ytick.labelsize': 13,
          'grid.color': 'k',
          'grid.linestyle': ':',
          'grid.linewidth': 0.8,
          'mathtext.fontset' : 'stix',
          'mathtext.rm'      : 'serif',
          'font.family'      : 'serif',
          'font.serif'       : "Times New Roman", # or "Times"          
         }
matplotlib.rcParams.update(params)

In [ ]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
output_size_network = len(classes)

In [ ]:
device = torch.device('cpu')

In [ ]:
trainloader_collection = [[CVAE_S1_MNIST_trainloader, trainloader_mnist_1], 
                          [CVAE_S2_MNIST_trainloader, trainloader_mnist_2], 
                          [CVAE_S3_MNIST_trainloader, trainloader_mnist_3]]
testloader_collection = [testloader_mnist_1, testloader_mnist_2, testloader_mnist_3]

In [ ]:
loadertypes = [0,1]
settings = [0, 1, 2]
epochs=150

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=output_size_network):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
    
model1 = ConvNet()
model1 = model1.to(device)

model2 = ConvNet()
model2 = model2.to(device)

model3 = ConvNet()
model3 = model3.to(device)

In [ ]:
model = [model1, model2, model3]

criterion = nn.CrossEntropyLoss()

optimizer1 = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
optimizer3 = optim.SGD(model3.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

optimizer = [optimizer1, optimizer2, optimizer3]

In [ ]:
for setting in zip(settings):
    setting = int(''.join(map(str, setting)))
    
    print('-------------------')
    if setting == 0:
        print('SETTING: Half-Split Imbalance')
    elif setting == 1:
        print('SETTING: MultiMajority')
    elif setting == 2:
        print('SETTING: MultiMinority')
    print('-------------------')
    
    for loadertype in zip(loadertypes):
        loadertype = int(''.join(map(str, loadertype)))
        
        if loadertype==0:
            print("CVAE Dataset")
            
        elif loadertype==1:
            print("Imbalanced Dataset")
        
        np.random.seed(setting)
        torch.manual_seed(setting)
        random.seed(setting)
        torch.cuda.manual_seed(setting)

        for epoch in range(1, epochs+1):  
            model[setting].train()
            for data, labels in trainloader_collection[setting][loadertype]:
                data, labels = data.to(device), labels.to(device)
                optimizer[setting].zero_grad()
                outputs = model[setting](data)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer[setting].step()

            print('Dataset: {} \tEpoch: {}'.format(loadertype, epoch))
    
            if setting == 0:
                print('Finished Training for setting', setting)
            elif setting == 1:
                print('Finished Training for setting', setting)
            elif setting == 2:
                print('Finished Training for setting', setting) 

-------------------
SETTING: Half-Split Imbalance
-------------------
CVAE Dataset
Dataset: 0 	Epoch: 1
Finished Training for setting 0
Dataset: 0 	Epoch: 2
Finished Training for setting 0
Dataset: 0 	Epoch: 3
Finished Training for setting 0
Dataset: 0 	Epoch: 4
Finished Training for setting 0
Dataset: 0 	Epoch: 5
Finished Training for setting 0
Dataset: 0 	Epoch: 6
Finished Training for setting 0
Dataset: 0 	Epoch: 7
Finished Training for setting 0
Dataset: 0 	Epoch: 8
Finished Training for setting 0
Dataset: 0 	Epoch: 9
Finished Training for setting 0
Dataset: 0 	Epoch: 10
Finished Training for setting 0
Dataset: 0 	Epoch: 11
Finished Training for setting 0
Dataset: 0 	Epoch: 12
Finished Training for setting 0
Dataset: 0 	Epoch: 13
Finished Training for setting 0
Dataset: 0 	Epoch: 14
Finished Training for setting 0
Dataset: 0 	Epoch: 15
Finished Training for setting 0
Dataset: 0 	Epoch: 16
Finished Training for setting 0
Dataset: 0 	Epoch: 17
Finished Training for setting 0
Dataset:

Dataset: 1 	Epoch: 1
Finished Training for setting 0
Dataset: 1 	Epoch: 2
Finished Training for setting 0
Dataset: 1 	Epoch: 3
Finished Training for setting 0
Dataset: 1 	Epoch: 4
Finished Training for setting 0
Dataset: 1 	Epoch: 5
Finished Training for setting 0
Dataset: 1 	Epoch: 6
Finished Training for setting 0
Dataset: 1 	Epoch: 7
Finished Training for setting 0
Dataset: 1 	Epoch: 8
Finished Training for setting 0
Dataset: 1 	Epoch: 9
Finished Training for setting 0
Dataset: 1 	Epoch: 10
Finished Training for setting 0
Dataset: 1 	Epoch: 11
Finished Training for setting 0
Dataset: 1 	Epoch: 12
Finished Training for setting 0
Dataset: 1 	Epoch: 13
Finished Training for setting 0
Dataset: 1 	Epoch: 14
Finished Training for setting 0
Dataset: 1 	Epoch: 15
Finished Training for setting 0
Dataset: 1 	Epoch: 16
Finished Training for setting 0
Dataset: 1 	Epoch: 17
Finished Training for setting 0
Dataset: 1 	Epoch: 18
Finished Training for setting 0
Dataset: 1 	Epoch: 19
Finished Traini

Dataset: 0 	Epoch: 1
Finished Training for setting 1
Dataset: 0 	Epoch: 2
Finished Training for setting 1
Dataset: 0 	Epoch: 3
Finished Training for setting 1
Dataset: 0 	Epoch: 4
Finished Training for setting 1
Dataset: 0 	Epoch: 5
Finished Training for setting 1
Dataset: 0 	Epoch: 6
Finished Training for setting 1
Dataset: 0 	Epoch: 7
Finished Training for setting 1
Dataset: 0 	Epoch: 8
Finished Training for setting 1
Dataset: 0 	Epoch: 9
Finished Training for setting 1
Dataset: 0 	Epoch: 10
Finished Training for setting 1
Dataset: 0 	Epoch: 11
Finished Training for setting 1
Dataset: 0 	Epoch: 12
Finished Training for setting 1
Dataset: 0 	Epoch: 13
Finished Training for setting 1
Dataset: 0 	Epoch: 14
Finished Training for setting 1
Dataset: 0 	Epoch: 15
Finished Training for setting 1
Dataset: 0 	Epoch: 16
Finished Training for setting 1
Dataset: 0 	Epoch: 17
Finished Training for setting 1
Dataset: 0 	Epoch: 18
Finished Training for setting 1
Dataset: 0 	Epoch: 19
Finished Traini

Dataset: 1 	Epoch: 2
Finished Training for setting 1
Dataset: 1 	Epoch: 3
Finished Training for setting 1
Dataset: 1 	Epoch: 4
Finished Training for setting 1
Dataset: 1 	Epoch: 5
Finished Training for setting 1
Dataset: 1 	Epoch: 6
Finished Training for setting 1
Dataset: 1 	Epoch: 7
Finished Training for setting 1
Dataset: 1 	Epoch: 8
Finished Training for setting 1
Dataset: 1 	Epoch: 9
Finished Training for setting 1
Dataset: 1 	Epoch: 10
Finished Training for setting 1
Dataset: 1 	Epoch: 11
Finished Training for setting 1
Dataset: 1 	Epoch: 12
Finished Training for setting 1
Dataset: 1 	Epoch: 13
Finished Training for setting 1
Dataset: 1 	Epoch: 14
Finished Training for setting 1
Dataset: 1 	Epoch: 15
Finished Training for setting 1
Dataset: 1 	Epoch: 16
Finished Training for setting 1
Dataset: 1 	Epoch: 17
Finished Training for setting 1
Dataset: 1 	Epoch: 18
Finished Training for setting 1
Dataset: 1 	Epoch: 19
Finished Training for setting 1
Dataset: 1 	Epoch: 20
Finished Train

Dataset: 0 	Epoch: 2
Finished Training for setting 2
Dataset: 0 	Epoch: 3
Finished Training for setting 2
Dataset: 0 	Epoch: 4
Finished Training for setting 2
Dataset: 0 	Epoch: 5
Finished Training for setting 2
Dataset: 0 	Epoch: 6
Finished Training for setting 2
Dataset: 0 	Epoch: 7
Finished Training for setting 2
Dataset: 0 	Epoch: 8
Finished Training for setting 2
Dataset: 0 	Epoch: 9
Finished Training for setting 2
Dataset: 0 	Epoch: 10
Finished Training for setting 2
Dataset: 0 	Epoch: 11
Finished Training for setting 2
Dataset: 0 	Epoch: 12
Finished Training for setting 2
Dataset: 0 	Epoch: 13
Finished Training for setting 2
Dataset: 0 	Epoch: 14
Finished Training for setting 2
Dataset: 0 	Epoch: 15
Finished Training for setting 2
Dataset: 0 	Epoch: 16
Finished Training for setting 2
Dataset: 0 	Epoch: 17
Finished Training for setting 2
Dataset: 0 	Epoch: 18
Finished Training for setting 2
Dataset: 0 	Epoch: 19
Finished Training for setting 2
Dataset: 0 	Epoch: 20
Finished Train

Dataset: 1 	Epoch: 3
Finished Training for setting 2
Dataset: 1 	Epoch: 4
Finished Training for setting 2
Dataset: 1 	Epoch: 5
Finished Training for setting 2
Dataset: 1 	Epoch: 6
Finished Training for setting 2
Dataset: 1 	Epoch: 7
Finished Training for setting 2
Dataset: 1 	Epoch: 8
Finished Training for setting 2
Dataset: 1 	Epoch: 9
Finished Training for setting 2
Dataset: 1 	Epoch: 10
Finished Training for setting 2
Dataset: 1 	Epoch: 11
Finished Training for setting 2
Dataset: 1 	Epoch: 12
Finished Training for setting 2
Dataset: 1 	Epoch: 13
Finished Training for setting 2
Dataset: 1 	Epoch: 14
Finished Training for setting 2
Dataset: 1 	Epoch: 15
Finished Training for setting 2
Dataset: 1 	Epoch: 16
Finished Training for setting 2
Dataset: 1 	Epoch: 17
Finished Training for setting 2
Dataset: 1 	Epoch: 18
Finished Training for setting 2
Dataset: 1 	Epoch: 19
Finished Training for setting 2
Dataset: 1 	Epoch: 20
Finished Training for setting 2
Dataset: 1 	Epoch: 21
Finished Trai

In [ ]:
global_accuracy = []
test_loss_hist = []
labels_list = []
pred_list= []

for setting in zip(settings):
    setting = int(''.join(map(str, setting)))
        
    test_loss = 0.0
    correct = 0
    total = 0
        
    model[setting].eval()
        
    with torch.no_grad():
        for data, labels in testloader_collection[setting]:
            images, labels = data.to(device), labels.to(device)
            output = model[setting](images)
    
            loss = criterion(output, labels)
            test_loss += loss.item()

            _, pred = torch.max(output, 1)    

            total += labels.size(0)
            correct += (pred == labels).sum().item()

            pred = pred.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()

            pred_list.append(pred)
            labels_list.append(labels)

    test_loss = test_loss/len(testloader_collection[setting])
    test_loss_hist.append(test_loss)
        
    accuracy = 100 * correct / total
    global_accuracy.append(accuracy)

In [ ]:
print('Test Loss for S1: {:.3f}'.format(test_loss_hist[0]))
print('Test Loss for S2: {:.3f}'.format(test_loss_hist[1]))
print('Test Loss for S3: {:.3f}'.format(test_loss_hist[2]))

Test Loss for S1: 0.039
Test Loss for S2: 0.024
Test Loss for S3: 0.028


In [ ]:
print('Accuracy for S1: {:.3f}%'.format(global_accuracy[0]))
print('Accuracy for S2: {:.3f}%'.format(global_accuracy[1]))
print('Accuracy for S3: {:.3f}%'.format(global_accuracy[2]))

Accuracy for S1: 98.978%
Accuracy for S2: 99.376%
Accuracy for S3: 98.968%


In [ ]:
slices = [4500, 7208, 872]

def imbalance_slicer(mylist):
    return [mylist[sum(slices[:i]):sum(slices[:i+1])] for i in range(len(slices))]

pred_list = [item for sublist in pred_list for item in sublist]
labels_list = [item for sublist in labels_list for item in sublist]

pred_model1 = imbalance_slicer(pred_list)[0]
pred_model2 = imbalance_slicer(pred_list)[1]
pred_model3 = imbalance_slicer(pred_list)[2]

labels_model1 = imbalance_slicer(labels_list)[0]
labels_model2 = imbalance_slicer(labels_list)[1]
labels_model3 = imbalance_slicer(labels_list)[2]

In [ ]:
pred_list = [pred_model1, pred_model2, pred_model3]
labels_list = [labels_model1, labels_model2, labels_model3]

f1_micro_list, f1_macro_list = [], []
gmean_micro_list, gmean_macro_list = [], []
bac_list, bac_adj_list = [], []
sens_micro_list, sens_macro_list = [], []
spec_micro_list, spec_macro_list = [], []
prec_micro_list, prec_macro_list = [], []
rec_micro_list, rec_macro_list = [], []

for setting in zip(settings):
    setting = int(''.join(map(str, setting)))
        
    f1_micro = f1_score(labels_list[setting], pred_list[setting], average='micro')
    f1_macro = f1_score(labels_list[setting], pred_list[setting], average='macro')
    f1_micro_list.append(f1_micro)
    f1_macro_list.append(f1_macro)
        
    gmean_micro = geometric_mean_score(labels_list[setting], pred_list[setting], average='micro')
    gmean_macro = geometric_mean_score(labels_list[setting], pred_list[setting], average='macro')
    gmean_micro_list.append(gmean_micro)
    gmean_macro_list.append(gmean_macro)
        
    bac = balanced_accuracy_score(labels_list[setting], pred_list[setting])
    bac_adj = balanced_accuracy_score(labels_list[setting], pred_list[setting], adjusted=True)
    bac_list.append(bac)
    bac_adj_list.append(bac_adj)
        
    sens_micro = sensitivity_score(labels_list[setting], pred_list[setting], average='micro')
    sens_macro = sensitivity_score(labels_list[setting], pred_list[setting], average='macro')
    sens_micro_list.append(sens_micro)
    sens_macro_list.append(sens_macro)
        
    spec_micro = specificity_score(labels_list[setting], pred_list[setting], average='micro')
    spec_macro = specificity_score(labels_list[setting], pred_list[setting], average='macro')
    spec_micro_list.append(sens_micro)
    spec_macro_list.append(sens_macro)
                                       
    prec_micro = precision_score(labels_list[setting], pred_list[setting], average='micro')
    prec_macro = precision_score(labels_list[setting], pred_list[setting], average='macro')
    prec_micro_list.append(prec_micro)
    prec_macro_list.append(prec_macro)
                                       
    rec_micro = recall_score(labels_list[setting], pred_list[setting], average='micro')
    rec_macro = recall_score(labels_list[setting], pred_list[setting], average='macro')
    rec_micro_list.append(rec_micro)
    rec_macro_list.append(rec_macro)

In [ ]:
metrics_list = [f1_micro_list, f1_macro_list, gmean_micro_list, gmean_macro_list, bac_list, bac_adj_list, sens_micro_list, sens_macro_list, spec_micro_list, spec_macro_list,
                prec_micro_list, prec_macro_list, rec_micro_list, rec_macro_list]

names = ["F1 Micro", "F1 Macro", "GMean Micro", "GMean Macro", "Balanced Accuracy", "Adjusted Balanced Accuracy", "Sensitivity Micro", "Sensitivity Macro", "Specificity Micro", 
            "Specificity Macro", "Precision Micro", "Precision Macro", "Recall Micro", "Recall Macro"]

for metric, name in zip(metrics_list, names):
    print('---------------')
    print('MODEL: S1')
    print('METRIC:', name)
    print('---------------')
    print(' {:.2f}'.format(metric[0]))
    
    print('---------------')
    print('MODEL: S2')
    print('METRIC:', name)
    print('---------------')
    print('{:.2f}'.format(metric[1]))
    
    print('---------------')
    print('MODEL: S3')
    print('METRIC:', name)
    print('---------------')
    print('{:.2f}'.format(metric[2]))
    
    print('\n')

---------------
MODEL: S1
METRIC: F1 Micro
---------------
 0.99
---------------
MODEL: S2
METRIC: F1 Micro
---------------
0.99
---------------
MODEL: S3
METRIC: F1 Micro
---------------
0.99


---------------
MODEL: S1
METRIC: F1 Macro
---------------
 0.99
---------------
MODEL: S2
METRIC: F1 Macro
---------------
0.98
---------------
MODEL: S3
METRIC: F1 Macro
---------------
0.92


---------------
MODEL: S1
METRIC: GMean Micro
---------------
 0.99
---------------
MODEL: S2
METRIC: GMean Micro
---------------
1.00
---------------
MODEL: S3
METRIC: GMean Micro
---------------
0.99


---------------
MODEL: S1
METRIC: GMean Macro
---------------
 0.99
---------------
MODEL: S2
METRIC: GMean Macro
---------------
0.98
---------------
MODEL: S3
METRIC: GMean Macro
---------------
0.95


---------------
MODEL: S1
METRIC: Balanced Accuracy
---------------
 0.98
---------------
MODEL: S2
METRIC: Balanced Accuracy
---------------
0.97
---------------
MODEL: S3
METRIC: Balanced Accuracy
---